In [53]:
import pandas as pd
from time import time
from pyelasticsearch import ElasticSearch



In [54]:
train_data_path = '/Users/abhilekh/Downloads/train_u6lujuX_CVtuZ9i.csv'

In [55]:
test_data_path = '/Users/abhilekh/Downloads/test_Y3wMUE5_7gLdaTN.csv'

In [56]:
train=pd.read_csv(train_data_path);
test=pd.read_csv(test_data_path);
print(train.shape);
print(test.shape)
train.head()


(614, 13)
(367, 12)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [57]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [63]:
CHUNKSIZE=100
index_name_train="loan_prediction_train"
index_name_test="loan_prediction_test"
doc_type_train="av-lp_train"
doc_type_test="av-lp_test"

In [93]:
#to get the data to elastic search
def index_data(data_path,chunksize,index_name, doc_type):
    f=open(data_path)
    csvfile=pd.read_csv(f, iterator=True, chunksize=CHUNKSIZE)
    es=ElasticSearch("http://localhost:9200/") #points to the elastic search instance
    try:        
        es.delete_index(index_name)
    except:
        pass
    es.create_index(index_name)
    
    for i,df in enumerate(csvfile):
        records=df.where(pd.notnull(df),None).T.to_dict()
        print(records)
        list_records=[records[it] for it in records]
        #print(list_records)
        try:
            es.bulk_index(index_name,doc_type,list_records)
        except :
            print("error!, skiping chunk!")
            pass
    
        
        

In [95]:
index_data(train_data_path, CHUNKSIZE, index_name_train, doc_type_train) # Indexing train data

index_data(test_data_path, CHUNKSIZE, index_name_test, doc_type_test) # Indexing test data


{0: {'Loan_ID': 'LP001002', 'Gender': 'Male', 'Married': 'No', 'Dependents': '0', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': 5849, 'CoapplicantIncome': 0, 'LoanAmount': None, 'Loan_Amount_Term': 360.0, 'Credit_History': 1.0, 'Property_Area': 'Urban', 'Loan_Status': 'Y'}, 1: {'Loan_ID': 'LP001003', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '1', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': 4583, 'CoapplicantIncome': 1508, 'LoanAmount': 128.0, 'Loan_Amount_Term': 360.0, 'Credit_History': 1.0, 'Property_Area': 'Rural', 'Loan_Status': 'N'}, 2: {'Loan_ID': 'LP001005', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '0', 'Education': 'Graduate', 'Self_Employed': 'Yes', 'ApplicantIncome': 3000, 'CoapplicantIncome': 0, 'LoanAmount': 66.0, 'Loan_Amount_Term': 360.0, 'Credit_History': 1.0, 'Property_Area': 'Urban', 'Loan_Status': 'Y'}, 3: {'Loan_ID': 'LP001006', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '0', 'Education': 'Not Graduat

{500: {'Loan_ID': 'LP002603', 'Gender': 'Female', 'Married': 'No', 'Dependents': '0', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': 645, 'CoapplicantIncome': 3683, 'LoanAmount': 113.0, 'Loan_Amount_Term': 480, 'Credit_History': 1.0, 'Property_Area': 'Rural', 'Loan_Status': 'Y'}, 501: {'Loan_ID': 'LP002606', 'Gender': 'Female', 'Married': 'No', 'Dependents': '0', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': 3159, 'CoapplicantIncome': 0, 'LoanAmount': 100.0, 'Loan_Amount_Term': 360, 'Credit_History': 1.0, 'Property_Area': 'Semiurban', 'Loan_Status': 'Y'}, 502: {'Loan_ID': 'LP002615', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '2', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': 4865, 'CoapplicantIncome': 5624, 'LoanAmount': 208.0, 'Loan_Amount_Term': 360, 'Credit_History': 1.0, 'Property_Area': 'Semiurban', 'Loan_Status': 'Y'}, 503: {'Loan_ID': 'LP002618', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '1', 'Educatio

{100: {'Loan_ID': 'LP001534', 'Gender': 'Male', 'Married': 'No', 'Dependents': '0', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': 4452, 'CoapplicantIncome': 0, 'LoanAmount': 131.0, 'Loan_Amount_Term': 360.0, 'Credit_History': 1.0, 'Property_Area': 'Rural'}, 101: {'Loan_ID': 'LP001542', 'Gender': 'Female', 'Married': 'Yes', 'Dependents': '0', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': 2262, 'CoapplicantIncome': 0, 'LoanAmount': None, 'Loan_Amount_Term': 480.0, 'Credit_History': 0.0, 'Property_Area': 'Semiurban'}, 102: {'Loan_ID': 'LP001547', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '1', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': 3901, 'CoapplicantIncome': 0, 'LoanAmount': 116.0, 'Loan_Amount_Term': 360.0, 'Credit_History': 1.0, 'Property_Area': 'Urban'}, 103: {'Loan_ID': 'LP001548', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '2', 'Education': 'Not Graduate', 'Self_Employed': 'No', 'ApplicantIncome': 268